In [ ]:
#Import bundle sampling and analysis functions
from dreimac import CircularCoords

from circle_bundles.viz.pca_viz import show_pca
from circle_bundles.viz.thumb_grids import *
from circle_bundles.viz.fiber_vis import *
from circle_bundles.viz.nerve_vis import *
from circle_bundles.viz.base_vis import *
from circle_bundles.viz.lattice_vis import *
from circle_bundles.viz.circle_vis import *

from circle_bundles.local_analysis import *
from circle_bundles.fiberwise_clustering import *
from circle_bundles.geometric_unwrapping import *
from circle_bundles.z2_linear import *

from circle_bundles.bundle import build_bundle, attach_bundle_viz_methods
from circle_bundles.covers import MetricBallCover, TriangulationStarCover
from circle_bundles.metrics import S1_dist2, get_dist_mat
from circle_bundles.local_triv import *
from circle_bundles.triangle_cover_builders import make_rp2_cover
from circle_bundles.nerve_utils import *
attach_bundle_viz_methods()

#For saving
import pickle

import pandas as pd

folder_path = '/Users/bradturow/Desktop/Diagrams/Clean_Diagrams/'


# Load The Dataset

In [ ]:
#Load the dataset of binary step-edge circles extracted from K_50_60
from optical_flow.contrast import *
from optical_flow.patch_viz import *
folder_path = '/Users/bradturow/Desktop/Circle Bundle Code'
file_name = 'K_50_60_Circles.pkl'
file_path = folder_path + file_name

with open(file_path, 'rb') as f:
    [data, C_circ] =pickle.load(f)

#Compute the predominant direction and directionality for each patch
predom_dirs, ratios = get_predominant_dirs(data)

#Create a visualizer for optical flow patches
patch_vis = make_patch_visualizer()

print(f'Sample contains {len(data)} high-contrast optical flow patches.')
print(f'A total of {len(C_circ)} clusters.')


In [ ]:
#Show a sample
j = 17
fig = show_data_vis(data[C_circ[j]], patch_vis, max_samples = 30)
plt.show()

In [ ]:
#View PCA of the dataset colored according to assigned circular cluster
#show_PCA(data, U= C_circ)
show_pca(data, U = C_circ)


In [ ]:
#Show correlations between circular coordinate and predominant direction on each circular component
from circle_bundles.viz.angles import *

#Run circular coordinates on the patches in each cluster 
to_show = [9, 11, 15]
n_landmarks = 100
prime = 17

circle_datasets = []
all_angles = []
titles = []

for j in to_show:
    indices = C_circ[j]
    circle_datasets.append(data[indices])
    cc = CircularCoords(data[indices], n_landmarks, prime=prime)
    all_angles.append(cc.get_coordinates())
    titles.append(f'Component {j}')

file_name = 'Sample_ccs_50_60.pdf'
save_path = folder_path + file_name

fig, axes = circle_vis_grid(
    circle_datasets,
    all_angles,
    patch_vis,
    titles=titles,
    per_circle=8,
    circle_radius=1.0,
    extent_factor=1.2,
    circle_figsize=5,
    circle_dpi=250,
    circle_zoom=0.13,
    circle_linewidth=1.0,
    circle_color="black",
    n_cols=3,
    title_fontsize=16,
    figsize_per_panel=5,
    fig_dpi=150,
    save_path=None,
)
plt.show()

#Show correlations between circular coordinate and predominant direction on each circular component
angle_arrays = []
to_compare = []
labels = []
empty_titles = []
for j, ang_arr in enumerate(all_angles):
    angle_arrays.append(ang_arr)
    indices = C_circ[to_show[j]]
    predoms = predom_dirs[indices]
    angle_arrays.append(predoms)
    if j == 0:
        labels.append(f'Circular Coordinate')
    else:
        labels.append('')
    labels.append(f'Predominant Flow Axis')
    empty_titles.append('')
    to_compare.append([2*j+1,2*j])



fig = compare_angle_pairs(
    angle_arrays,
    to_compare,
    x_range = (0,np.pi),
    labels = labels,
    align = False,
    s = 1.0,
    fontsize = 14,
    ncols = 3,
    titles = empty_titles,
    titlesize = 20,
    show_metrics = False)
plt.show()




## Lifting The Step Edge Circles

In [ ]:
#Compute lifted predominant angles in S1
lifted_base_points = get_lifted_predom_dirs(data, eps=1e-12)

n_landmarks = 16
lmk_angs = np.linspace(0, 2*np.pi,n_landmarks, endpoint= False)
landmarks = np.array([np.cos(lmk_angs), np.sin(lmk_angs)]).T
overlap = 1.5
radius = overlap* np.pi/n_landmarks

lifted_cover = MetricBallCover(lifted_base_points, landmarks, radius, metric = S1_dist2)
lifted_cover_data = lifted_cover.build()
lifted_summ = lifted_cover.summarize(plot = True)

In [ ]:
#Try running circular coordinates on a single open set
j= 6
n_local_landmarks = 200
prime = 43
fiber_data = data[lifted_cover.U[j]]
patch_angles, n_warnings, n_lmks = compute_circular_coords_dreimac(fiber_data, 
                                                                   n_landmarks_init = n_local_landmarks, 
                                                                   CircularCoords_cls = CircularCoords)

print(n_warnings, n_lmks)
show_pca(fiber_data, colors = patch_angles)




In [ ]:
#Compute local trivializations and characteristic classes
lifted_bundle = build_bundle(
    data,
    lifted_cover,
    CircularCoords_cls=CircularCoords
    show=True)


In [ ]:
fig = lifted_bundle.compare_trivs()
plt.show()

In [ ]:
triv_result = lifted_bundle.get_global_trivialization()
fiber_angles = triv_result.F
print('Global coordinates computed.')

In [ ]:
#Show a sample of coordinatized data

base_angles = np.arctan2(lifted_cover.base_points[:,1], lifted_cover.base_points[:,0]) % (2*np.pi)
coords = np.array([base_angles, fiber_angles]).T
file_name = 'Recovered_Lifted_Step_Edge_Torus.pdf'
save_path = folder_path + file_name
save_path = None
fig = lattice_vis(data, coords, patch_vis, 
                             per_row=7,
                              per_col = 9,
                             figsize=16, save_path = save_path)
plt.show()


In [ ]:
#Show an interactive visualization of the bundle structure
app = lifted_bundle.show_bundle(colors = fiber_angles)
plt.show()

In [ ]:
#Show an interactive visualization of the bundle structure

from circle_bundles.viz.bundle_dash import *
base_points = np.array([np.cos(2*predom_dirs), np.sin(2*predom_dirs)]).T

app = show_bundle_vis(
    base_points = base_points,
    data = data,
    base_metric = None,
    initial_r = 0.1,
    r_max = 2.0,
    colors = fiber_angles,
    landmark_inds = None,
    max_samples = 10_000)
plt.show()